# Medical Imaging Expert Agent
In this notebook we create the medical imaging expert sub-agent

#### Ensure the latest version of boto3 is shown below

In [ ]:
!pip freeze | grep boto3

#### Load in environment variables to notebook

In [ ]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r account_id
%store -r region

# Retrieve model lists
%store -r agent_foundation_model

#### Retrieve imports environment variable and bring libraries into notebook

In [ ]:
%run $IMPORTS_PATH

# Prerequisites

This notebook assumes that you have deployed the CloudFormation stack located at https://github.com/aws-samples/amazon-bedrock-agents-cancer-biomarker-discovery to your AWS account in workshop mode.

# Agent Creation
In this section we create the sub-agent

#### Define agent configuration below 

In [ ]:
agent_name = 'Medical-imaging-expert'
agent_description = "CT scan analysis"
agent_instruction = """You are a medical research assistant AI specialized in processing medical imaging scans of 
patients. Your primary task is to create medical imaging jobs, or provide relevant medical insights after the 
jobs have completed execution. Use only the appropriate tools as required by the specific question. Follow these 
instructions carefully: 1. For computed tomographic (CT) lung imaging biomarker analysis: a. Identify the patient 
subject ID(s) based on the conversation. b. Use the compute_imaging_biomarker tool to trigger the long-running job,
passing the subject ID(s) as an array of strings (for example, ["R01-043", "R01-93"]). c. Only if specifically asked 
for an analysis, use the analyze_imaging_biomarker tool to process the results from the previous job. 
. When providing your response: a. Start with a brief summary of your understanding of the user's query. 
b. Explain the steps you're taking to address the query. Ask for clarifications from the user if required. 
c. Present the results of the medical imaging jobs if complete.""" 

#### Instantiate agent with the desired configuration

In [ ]:
agents = AgentsForAmazonBedrock()

medical_imaging_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

medical_imaging_agent

#### Extract useful agent information

In [ ]:
medical_imaging_agent_id = medical_imaging_agent[0]
medical_imaging_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{medical_imaging_agent_id}"

medical_imaging_agent_id, medical_imaging_agent_arn

#### Define function details for ActionGroup

In [ ]:
function_defs = [
    {
      "name": "compute_imaging_biomarker",
      "description": "compute the imaging biomarker features from lung CT scans within the tumor for a list of patient subject IDs",
      "parameters": {
        "subject_id": {
          "description": "an array of strings representing patient subject IDs, example ['R01-222', 'R01-333']",
          "required": True,
          "type": "array"
        }
      },
      "requireConfirmation": "DISABLED"
    },
    {
      "name": "analyze_imaging_biomarker",
      "description": "analyze the result imaging biomarker features from lung CT scans within the tumor for a list of patient subject ID",
      "parameters": {
        "subject_id": {
          "description": "an array of strings representing patient subject IDs, example ['R01-222', 'R01-333']",
          "required": True,
          "type": "array"
        }
      },
      "requireConfirmation": "DISABLED"
    }
]

#### Attach Lambda function and create ActionGroup
Note: This uses the default Lambda function name "imaging-biomarker-lambda", this could be different in your account so double-check that this function exists and if not change the lambda_function_name in the code below

In [ ]:
# Define Lambda func. details, hardcode Lambda function name
medical_imaging_lambda_function_name = "imaging-biomarker-lambda"  # Change if different in your account
medical_imaging_lambda_function_arn = f"arn:aws:lambda:{region}:{account_id}:function:{medical_imaging_lambda_function_name}"
%store medical_imaging_lambda_function_arn

In [ ]:
agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=medical_imaging_lambda_function_name,
    source_code_file=medical_imaging_lambda_function_arn,
    agent_action_group_name="imagingBiomarkerProcessing",
    agent_action_group_description="Actions for processing imaging biomarker within CT scans for a list of subjects",
    agent_functions=function_defs,
    verbose=True
)

#### Add resource based policy to Lambda function to allow agent to invoke

In [ ]:
lambda_client = boto3.client('lambda', region)

# Define the resource policy statement
policy_statement = {
    "Sid": "AllowBedrockAgentAccess",
    "Effect": "Allow",
    "Principal": {
        "Service": "bedrock.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": medical_imaging_lambda_function_arn,
    "Condition": {
        "ArnEquals": {
            "aws:SourceArn": medical_imaging_agent_arn
        }
    }
}

try:
    # Get the current policy
    response = lambda_client.get_policy(FunctionName=medical_imaging_lambda_function_arn)
    current_policy = json.loads(response['Policy'])
    
    # Add the new statement to the existing policy
    current_policy['Statement'].append(policy_statement)
    
except lambda_client.exceptions.ResourceNotFoundException:
    # If there's no existing policy, create a new one
    current_policy = {
        "Version": "2012-10-17",
        "Statement": [policy_statement]
    }

# Convert the policy to JSON string
updated_policy = json.dumps(current_policy)

# Add or update the resource policy
response = lambda_client.add_permission(
    FunctionName=medical_imaging_lambda_function_arn,
    StatementId="AllowMedicalImagingAgentAccess",
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=medical_imaging_agent_arn
)

print("Resource policy added successfully.")
print("Response:", response)

#### Invoke Medical Imaging Expert Agent Test Alias to see that it answers question properly
Note: This agent needs a supervisor agent to properly answer some questions

In [ ]:
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)

session_id = str(uuid.uuid1())


test_query = "Can you compute the imaging biomarkers for the 2 patients with the lowest gdf15 expression values?"

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=medical_imaging_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

#### Now that agent has been tested via direct invoke, prepare it by creating an alias

In [ ]:
medical_imaging_agent_alias_id, medical_imaging_agent_alias_arn = agents.create_agent_alias(
    medical_imaging_agent[0], 'v1'
)

%store medical_imaging_agent_alias_arn
medical_imaging_agent_alias_id, medical_imaging_agent_alias_arn